In [1]:
# Read in data
hmeq = read.csv("C:/Users/student/Desktop/Senior/MATH 421 - R/Assignment/hmeq.csv")
str(hmeq)  #5960 obs with 13 variables

'data.frame':	5960 obs. of  13 variables:
 $ BAD    : int  1 1 1 1 0 1 1 1 1 1 ...
 $ LOAN   : int  1100 1300 1500 1500 1700 1700 1800 1800 2000 2000 ...
 $ MORTDUE: num  25860 70053 13500 NA 97800 ...
 $ VALUE  : num  39025 68400 16700 NA 112000 ...
 $ REASON : Factor w/ 3 levels "","DebtCon","HomeImp": 3 3 3 1 3 3 3 3 3 3 ...
 $ JOB    : Factor w/ 7 levels "","Mgr","Office",..: 4 4 4 1 3 4 4 4 4 6 ...
 $ YOJ    : num  10.5 7 4 NA 3 9 5 11 3 16 ...
 $ DEROG  : int  0 0 0 NA 0 0 3 0 0 0 ...
 $ DELINQ : int  0 2 0 NA 0 0 2 0 2 0 ...
 $ CLAGE  : num  94.4 121.8 149.5 NA 93.3 ...
 $ NINQ   : int  1 0 1 NA 0 1 1 0 1 0 ...
 $ CLNO   : int  9 14 10 NA 14 8 17 8 12 13 ...
 $ DEBTINC: num  NA NA NA NA NA ...


In [2]:
#change target variable type
hmeq$BAD=as.factor(hmeq$BAD)
sum(is.na(hmeq)) #there are 4740 missing values

[1] 4740

In [3]:
#quick clean the data by deleting obs with missing value
hmeq[hmeq=='']=NA
#complete.cases function only keeps obs without missing
data=hmeq[complete.cases(hmeq),]

In [4]:
#change target variable "BAD" to "Target"
names(data)[1]='TARGET'
table(data$TARGET) #only 300 obs repaid loan - imbalanced dataset 


   0    1 
3064  300 

In [5]:
#load in required package
library(caret)
library(ranger)

Loading required package: lattice
Loading required package: ggplot2


In [6]:
#split data to 70% training and 30% testing
set.seed(2018)
splitIndex=createDataPartition(data$TARGET, p=.70, list=FALSE, times=1)
train=data[splitIndex,]
test=data[-splitIndex,]

In [7]:
#run model on original dataset
forest1=ranger(TARGET~.,data=train)
pred=predict(forest1,data=test)$predictions
cm=confusionMatrix(pred,test$TARGET,positive="1")
cm$overall['Accuracy']
orgAcc = cm$byClass['Balanced Accuracy']

prop.table(table(train$TARGET))

Accuracy 
0.9464817


         0          1 
0.91082803 0.08917197 

In [9]:
#undersampling
#train1 contians default loans
train1=train[train$TARGET=="1",]
n1=nrow(train1)
table(train1$TARGET)

#train2 contians repaid loans
train0=train[train$TARGET=="0",]
n0=nrow(train0)
table(train0$TARGET)

train00=train0[sample(1:n0,n1),]
train_under=rbind(train00,train1)


  0   1 
  0 210 


   0    1 
2145    0 

In [11]:
#test model using undersampled  dataset
model_under=ranger(TARGET~.,data=train_under)
pred_under=predict(model_under,data=test)$predictions
cm_under=confusionMatrix(pred_under,test$TARGET,positive="1")
underAcc = cm_under$byClass['Balanced Accuracy']
underAcc

Balanced Accuracy 
        0.8009793

In [13]:
#oversampling
train11 = train1[sample(1:n1, n0, replace = TRUE), ]
train_over = rbind(train11, train0)
#test model using oversampled dataset
model_over=ranger(TARGET~.,data=train_over)
pred_over=predict(model_over,data=test)$predictions
cm_over=confusionMatrix(pred_over,test$TARGET,positive="1")
overAcc = cm_over$byClass['Balanced Accuracy']
overAcc

Balanced Accuracy 
        0.7655785

In [17]:
# Quick_bal: return balanced dataset
Quick_bal = function(data, method)
{
  if(method=="over"){
    
    set.seed(2018)
    splitIndex=createDataPartition(data$TARGET, p=.70, list=FALSE, times=1)
    train=data[splitIndex,]
    test=data[-splitIndex,]
    
    train1=train[train$TARGET=="1",]
    n1=nrow(train1)
    table(train1$TARGET)
    
    train0=train[train$TARGET=="0",]
    n0=nrow(train0)
    table(train0$TARGET)
    
    train11 = train1[sample(1:n1, n0, replace = TRUE), ]
    balanced = rbind(train11, train0)
    
  }
   else if(method=="under"){
    
    set.seed(2018)
    splitIndex=createDataPartition(data$TARGET, p=.70, list=FALSE, times=1)
    train=data[splitIndex,]
    test=data[-splitIndex,]
    
    train1=train[train$TARGET=="1",]
    n1=nrow(train1)
    table(train1$TARGET)
    
    train0=train[train$TARGET=="0",]
    n0=nrow(train0)
    table(train0$TARGET)
    
    train00=train0[sample(1:n0,n1),]
    balanced =rbind(train00,train1)
    
  }
  else {
    print("Please pick under or over sampling method")
  }
}

In [18]:
#testing to make sure it works
balanced = Quick_bal(data,"over")
nrow(balanced)

[1] 4290

In [19]:
#Quick_model_bal function return balanced accuracy after data being balanced
Quick_model_bal = function (data, method){
  balanced = Quick_bal(data, method)
  model=ranger(TARGET~.,data=balanced)
  pred=predict(model,data=test)$predictions
  cm=confusionMatrix(pred,test$TARGET,positive="1")
  Acc = cm$byClass['Balanced Accuracy']
  Acc
}

In [20]:
#testing 
Quick_model_bal (data,"over")

Balanced Accuracy 
        0.7322452